In [62]:
import sys
import os
workspace_path = os.path.join(os.path.abspath(".."))
sys.path.append(workspace_path)
workspace_path

'c:\\Users\\Kajsa\\VSCodeProjects\\job_discrimination_sandbox'

In [63]:
import pandas as pd
from pandas.core.frame import DataFrame
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import glob
import re
import warnings
warnings.simplefilter("ignore")
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split, cross_val_score, ShuffleSplit, RepeatedKFold
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn import tree, preprocessing
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay

# Local imports
from src.helpers import *

In [64]:
pd.options.display.float_format = "{:.2f}".format

In [65]:
path_to_cleaned_data = os.path.join(workspace_path, "data", "cleaned_data")
file_dir = os.path.join(path_to_cleaned_data, "Job_Bulletins", "labeled", "*")
file_paths = glob.glob(file_dir)

In [66]:
# Reading application statistics into dataframe
file_path = os.path.join(path_to_cleaned_data, "bulletins_w_labels_and_content.csv")
df = pd.read_csv(file_path, dtype={"ID": str})

In [67]:
df

,ID,Job Description,Apps Received,Female,Male,Unknown_Gender,File Names,Label 60/40,Numeric label 60/40,Label 70/30,Numeric label 70/30,Cleaned text,Text
0,9206,311 DIRECTOR,54,20,31,3,311 DIRECTOR 9206 041814.txt,M,2,N,0,director class code open date annual salary du...,311 DIRECTOR Class Code: 9206 Open Date:...
1,1223,ACCOUNTING CLERK,648,488,152,8,ACCOUNTING CLERK 1223 071318.txt,W,1,W,1,accounting clerk class code open date exam ope...,ACCOUNTING CLERK Class Code: 1223 Open ...
2,7260,AIRPORT MANAGER,51,13,37,1,AIRPORT MANAGER 7260 120216.txt,M,2,M,2,airport manager class code open date exam open...,AIRPORT MANAGER Class Code: 7260 Open D...
3,3227,AIRPORT POLICE LIEUTENANT,48,9,38,1,AIRPORT POLICE LIEUTENANT 3227 091616.txt,M,2,M,2,airport police lieutenant class code open date...,AIRPORT POLICE LIEUTENANT ...
4,2400,AQUARIST,40,15,24,1,AQUARIST 2400 050214.txt,M,2,N,0,aquarist class code open date annual salary ca...,AQUARIST Class Code: 2400 Open Date: 05...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,7840,WASTEWATER TREATMENT LABORATORY MANAGER,16,6,9,1,WASTEWATER TREATMENT LABORATORY MANAGER 7840 1...,M,2,N,0,wastewater treatment laboratory manager class ...,WASTEWATER TREATMENT LABORATORY MANAGER Class...
173,4123,WASTEWATER TREATMENT OPERATOR,125,9,113,3,WASTEWATER TREATMENT OPERATOR 120718.txt,M,2,M,2,wastewater treatment operator class code open ...,WASTEWATER TREATMENT OPERATOR Class Code: ...
174,7857,WATER MICROBIOLOGIST,179,89,82,8,WATER MICROBIOLOGIST 7857 072514 rev073114.txt,N,0,N,0,water microbiologist class code open date revi...,WATER MICROBIOLOGIST Class Code: 7857...
175,3912,WATER UTILITY WORKER,96,2,92,2,WATER UTILITY WORKER 3912 120817.txt,M,2,M,2,water utility worker class code open date exam...,WATER UTILITY WORKER Class Code: 3912 Op...


In [68]:
df.drop(columns=["Label 60/40", "Numeric label 60/40", "Label 70/30", "Numeric label 70/30", "Text", "File Names"], inplace=True)

In [69]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177 entries, 0 to 176
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ID               177 non-null    object
 1   Job Description  177 non-null    object
 2   Apps Received    177 non-null    int64 
 3   Female           177 non-null    int64 
 4   Male             177 non-null    int64 
 5   Unknown_Gender   177 non-null    int64 
 6   Cleaned text     177 non-null    object
dtypes: int64(4), object(3)
memory usage: 9.8+ KB


In [70]:
df["Male share"] = df["Male"] / df["Apps Received"]
df["Female share"] = df["Female"] / df["Apps Received"]
df["Unknown share"] = df["Unknown_Gender"] / df["Apps Received"]

In [71]:
df

,ID,Job Description,Apps Received,Female,Male,Unknown_Gender,Cleaned text,Male share,Female share,Unknown share
0,9206,311 DIRECTOR,54,20,31,3,director class code open date annual salary du...,0.57,0.37,0.06
1,1223,ACCOUNTING CLERK,648,488,152,8,accounting clerk class code open date exam ope...,0.23,0.75,0.01
2,7260,AIRPORT MANAGER,51,13,37,1,airport manager class code open date exam open...,0.73,0.25,0.02
3,3227,AIRPORT POLICE LIEUTENANT,48,9,38,1,airport police lieutenant class code open date...,0.79,0.19,0.02
4,2400,AQUARIST,40,15,24,1,aquarist class code open date annual salary ca...,0.60,0.38,0.03
...,...,...,...,...,...,...,...,...,...,...
172,7840,WASTEWATER TREATMENT LABORATORY MANAGER,16,6,9,1,wastewater treatment laboratory manager class ...,0.56,0.38,0.06
173,4123,WASTEWATER TREATMENT OPERATOR,125,9,113,3,wastewater treatment operator class code open ...,0.90,0.07,0.02
174,7857,WATER MICROBIOLOGIST,179,89,82,8,water microbiologist class code open date revi...,0.46,0.50,0.04
175,3912,WATER UTILITY WORKER,96,2,92,2,water utility worker class code open date exam...,0.96,0.02,0.02


In [72]:
X = df["Cleaned text"]
y = df[["Male share", "Female share", "Unknown share"]]

In [73]:
# Initializing a vectorizer in order to transform the text to numbers

vect = TfidfVectorizer()

In [74]:
# Transforming X to tf-idf-vectors
X = vect.fit_transform(X).todense()

In [79]:
X.shape

(177, 3835)

In [55]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1000)

ValueError: Found input variables with inconsistent numbers of samples: [1, 177]

In [ ]:
model = DecisionTreeRegressor(max_depth=4)

In [ ]:
model.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=4)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_pred

array([[0.95800667, 0.02547363, 0.0165197 ],
       [0.71119521, 0.26291233, 0.02589246],
       [0.95800667, 0.02547363, 0.0165197 ],
       [0.95800667, 0.02547363, 0.0165197 ],
       [0.44156687, 0.53625842, 0.02217471],
       [0.95800667, 0.02547363, 0.0165197 ],
       [0.21644933, 0.7670114 , 0.01653927],
       [0.95800667, 0.02547363, 0.0165197 ],
       [0.44156687, 0.53625842, 0.02217471],
       [0.71119521, 0.26291233, 0.02589246],
       [0.44156687, 0.53625842, 0.02217471],
       [0.6       , 0.375     , 0.025     ],
       [0.50075887, 0.47559649, 0.02364464],
       [0.71119521, 0.26291233, 0.02589246],
       [0.71119521, 0.26291233, 0.02589246],
       [0.95800667, 0.02547363, 0.0165197 ],
       [0.71119521, 0.26291233, 0.02589246],
       [0.71119521, 0.26291233, 0.02589246],
       [0.50075887, 0.47559649, 0.02364464],
       [0.95800667, 0.02547363, 0.0165197 ],
       [0.71119521, 0.26291233, 0.02589246],
       [0.93033811, 0.04798318, 0.02167871],
       [0.

In [ ]:
for line in y_pred:
    print(sum(line))

0.9999999999999997
0.9999999999999999
0.9999999999999997
0.9999999999999997
0.9999999999999999
0.9999999999999997
0.9999999999999999
0.9999999999999997
0.9999999999999999
0.9999999999999999
0.9999999999999999
1.0
1.0
0.9999999999999999
0.9999999999999999
0.9999999999999997
0.9999999999999999
0.9999999999999999
1.0
0.9999999999999997
0.9999999999999999
1.0
1.0
1.0
1.0
0.9999999999999999
1.0
1.0
1.0
0.9999999999999999
1.0000000000000002
1.0
1.0
1.0000000000000002
0.9999999999999997
1.0
0.9999999999999997
1.0
0.9999999999999999
0.9999999999999997
1.0000000000000002
0.9999999999999999
0.9999999999999997
1.0
1.0


In [ ]:
resultsdf = X_test
resultsdf

matrix([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.04093129, 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ]])

In [ ]:
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

In [ ]:
n_scores = cross_val_score(model, X, y, scoring="neg_mean_absolute_error", cv=cv, n_jobs=-1)

In [ ]:
from numpy import absolute, mean, std
n_scores = absolute(n_scores)
print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))